<a href="https://colab.research.google.com/github/ianwong74/Tweeter_Airline_Sentiment_Analysis/blob/main/csda_1040_lab2_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSDA1040 - Group 5 - Lab2 - US Airline Tweets Sentiment Analysis
** This will take up to 1 minute for initial load, Thank you for your patience*

In [1]:
# Comment these block out before publishing to Binder, packages listed in requirements.txt will be preinstalled when building the image
from IPython.utils import io

with io.capture_output() as captured: 
  !pip install html2text
  !pip install nltk
  !pip install db-sqlite3
  !pip install fastai==1.0.61
  !pip install pytest-shutil
  !pip install voila

In [2]:
import sqlite3 as db
import re
import os
import codecs
import pandas as pd
import numpy as np
import html2text
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import shutil
import functools
import time
import warnings
import urllib.request
warnings.filterwarnings('ignore') 
np.set_printoptions(precision=6, suppress=True)
import random
import ipywidgets as widgets
from IPython.display import display

import fastai
from fastai import *
from fastai.text import *
from fastai.core import *
import pickle
from fastai.callbacks import *

In [3]:
# download pkl files from shared file from Google drive
# CSDA 1040 Team, please update the shared file id if you want to use your train result. Otherwise you are using the team's master version.
# sentiments1.pkl: https://drive.google.com/file/d/1--FqAwtWYYWglNrA-NZ4bd_wMjp27Rg_/view?usp=sharing

from IPython.utils import io
with io.capture_output() as captured: 
  !gdown --id '1--FqAwtWYYWglNrA-NZ4bd_wMjp27Rg_' --output pkl

# tweets_main.db: https://drive.google.com/file/d/1F3wvsn2eAVT964uFit8iLqZipIBowyj7/view?usp=sharing
from IPython.utils import io
with io.capture_output() as captured: 
  !gdown --id '1F3wvsn2eAVT964uFit8iLqZipIBowyj7' --output tw
cn = db.connect('tw')
cmd = cn.cursor()

In [4]:
df = pd.read_sql('select tweet_body from tweets', cn) 
df.head()

,tweet_body
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...


In [5]:
learn = load_learner('.','pkl')
ts = pd.read_sql('select tweet_body from tweets', cn)
n = random.randint(0,len(ts)-1)
t = ts.loc[n]
print(t)
p = learn.predict(t)
print(p)

tweet_body    @SouthwestAir will you please tell me if I'll ...
Name: 6363, dtype: object
(Category tensor(0), tensor(0), tensor([0.9413, 0.0587]))


In [92]:
# define class & properties
class Sentiments():
    # class init function, load model jester_jokes (jj) from exported model from shared Google drive prepared by csda_1040_lab1.ipynb
    def __init__(self):
      # load mode to learner
      self.__learn = load_learner('.','pkl')
      # load tweets_main.db db-sqlite3 file, extract tweet_body (text)
      self.__tweets = pd.read_sql('select tweet_body from tweets', cn)
      n = random.randint(0,len(self.__tweets)-1)
      self.__get_preds(n)
 
    def __get_preds(self, n):
      self.__tweet_body = self.__tweets.loc[n]
      pred,_,_ = self.__learn.predict(self.__tweet_body)
      thumbs_up = '\U0001F44D'
      thumbs_down = '\U0001F44E'
      if str(pred) == '1':
        pred_rslt = thumbs_up
      else: 
        pred_rslt = thumbs_down
      self.__preds = pred_rslt
      
    def __next_preds(self):
      n = random.randint(0,len(self.__tweets)-1)
      self.__get_preds(n)



In [93]:
sentiments = Sentiments()
#sentiments._Sentiments__tweets.head()
print(sentiments._Sentiments__preds)
print(sentiments._Sentiments__tweet_body.values)

👎
['@JetBlue i am here - what is your resolution?']


In [100]:
sentiments._Sentiments__next_preds()
print(sentiments._Sentiments__preds)
print(sentiments._Sentiments__tweet_body.values)

👎
['@AmericanAir FYI...call stilling getting dropped.  After an hour of continuous dialing. Attempted to Cancelled Flight online but not able to. HELP!!!']


In [8]:
# create the recommender
# from io import StringIO
jester = Jester_Jokes()
ui_cold_start_container = None
ui_cold_start_sliders = None
ui_html = None
ui_slider = None
ui_pred = None
ui_confidence_bar = None
ui_confidence_lbl = None
ui_rated_bar = None
ui_rated_lbl = None
ui_sumbit = None
ui_container = None

def ftn_create_joke(joke, predicted_rating=0, show_prediction=True):
  # creates UI to display and rate one joke and returns it
  if(joke is not None): joke_text = jester.joke_text(joke)
  else:                 joke_text = ""

  #https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html

  s = "<br><div style='font-size:15px;' align='center'>" + joke_text + "</div><br>" 
  html = widgets.HTML(value=s)
  html.layout = widgets.Layout(border="none", width = '80%', height='auto')
  slider = widgets.FloatSlider(description='Your Rating:', value=0, min=-10, max=10, step=0.01, readout=True, readout_format='.2f')
  slider.tag = joke
  slider.style.handle_color = 'lightblue'
  pbar = widgets.FloatSlider(description='Predicted  : ', value=predicted_rating, min=-10, max=10, step=0.01, readout=True, readout_format='.2f', style={'bar_color': 'lightblue'}, disabled=True)                   
  items = [slider]
  if show_prediction: items.append(pbar)
  grid = widgets.GridBox(children=items)
  grid.layout = widgets.Layout(grid_template_columns='auto', width='auto')
  box = widgets.VBox([html, grid])
  box.layout = widgets.Layout(align_items="center", width='auto')
  return box, slider
  
def ftn_render_cold_start():
  # renders the cold-start UI, x jokes, 1 submit, no predictions (x number of jokes to learn before recommendation)
  # start with 10% of the jokes
  global ui_cold_start_container
  global ui_cold_start_sliders
  # list init
  jokes = []
  children = []
  sliders = []
  x = 10
  for i in range(0,x):
    n = random.randint(1,100)
    jokes.append(n)
  # add x jokes for initial rating
  for joke in jokes:
    container, slider = ftn_create_joke(joke=joke, show_prediction=False)
    children.append(container)
    sliders.append(slider)
  btn = widgets.Button(description="Rate")
  btn.layout.margin = '20px'
  btn.on_click(ftn_on_click_cold_start)
  children.append(btn)
  box = widgets.VBox(children)
  box.layout = widgets.Layout(align_items='center', width='auto')
  display(box)
  ui_cold_start_container = box
  ui_cold_start_sliders = sliders
  
def ftn_on_click_cold_start(btn):
  # action on first submission 
  global ui_cold_start_container
  global ui_container
  for slider in ui_cold_start_sliders:
    jester.add_rating(slider.tag, slider.value)
  ui_cold_start_container.close()
  del ui_cold_start_container
  ui_container.layout.visibility = 'visible'
  ftn_on_click_recommender(None)

def ftn_render_recommender():
  global ui_html
  global ui_slider
  global ui_pred
  global ui_confidence_bar
  global ui_confidence_lbl
  global ui_rated_bar
  global ui_rated_lbl
  global ui_sumbit
  global ui_container
  container, ui_slider = ftn_create_joke(None, 0)
  ui_html = container.children[0]
  ui_slider = container.children[1].children[0]
  ui_pred = container.children[1].children[1] 
  btn = widgets.Button(description='Rate')
  btn.layout.margin = '20px'
  btn.on_click(ftn_on_click_recommender)
  ui_sumbit = btn
  ui_confidence_bar = widgets.FloatProgress(description='Confidence Level:', value=jester.confidence, min=0, max=1, style={'bar_color': 'lightblue'})
  ui_confidence_lbl = widgets.Label('{:.0%}'.format(jester.confidence) + ' confidence in prediction accuracy')
  ui_rated_bar = widgets.IntProgress(description='Jokes Rated:', value=(100 - jester.unrated_jokes), min=0, max=100, style={'bar_color': 'lightblue'})
  ui_rated_lbl = widgets.Label('You have rated ' + str(100 - jester.unrated_jokes) + '/100 jokes')
  items = [ui_confidence_bar, ui_confidence_lbl, ui_rated_bar, ui_rated_lbl]
  grid = widgets.GridBox(children=items)
  grid.layout = widgets.Layout(grid_template_columns='auto', grid_gap='20px', width='auto', padding='5px', border='none')
  box = widgets.VBox([container, btn, grid])
  box.layout = widgets.Layout(align_items='center', padding='10px', width='auto')
  display(box)
  box.layout.visibility = 'hidden'
  ui_container = box

def ftn_on_click_recommender(btn):
  global ui_html
  global ui_slider
  global ui_pred
  global ui_confidence_bar
  global ui_confidence_lbl
  global ui_rated_bar
  global ui_rated_lbl
  global ui_sumbit
  if(btn is not None): jester.add_rating(ui_slider.tag, ui_slider.value)
  joke, pred = jester.next_joke()
  if joke is not None: joke_text = jester.joke_text(joke) 
  else: joke_test = 'Complete rating all jokes'
  ui_html.value = '<br><div style=''font-size:15px;'' align=''center''>' + joke_text + '</div><br>'
  ui_slider.tag = joke
  ui_slider.value = 0
  if(pred is not None): ui_pred.value = pred
  ui_confidence_bar.value = jester.confidence
  ui_confidence_lbl.value = '{:.0%}'.format(jester.confidence) + ' confidence in prediction accuracy'
  ui_rated_bar.value = (100 - jester.unrated_jokes)
  ui_rated_lbl.value = 'Rated ' + str(100 - jester.unrated_jokes) + ' out of 100 jokes'
  ui_sumbit.disabled = (joke is None)

NameError: ignored

In [ ]:
ftn_render_cold_start()
ftn_render_recommender()